In [1]:
## Imports
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input/nvidiaapex/repository/NVIDIA-apex-39e153a"))

['.gitignore', 'LICENSE', 'tests', 'setup.py', '.nojekyll', 'README.md', 'docs', 'csrc', 'examples', 'apex']


In [2]:
# Installing Nvidia Apex
! pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ../input/nvidiaapex/repository/NVIDIA-apex-39e153a

/opt/conda/lib/python3.6/site-packages/pip/_internal/commands/install.py:207: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-raea4ug7
Created temporary directory: /tmp/pip-req-tracker-fm4u4rl0
Created requirements tracker '/tmp/pip-req-tracker-fm4u4rl0'
Created temporary directory: /tmp/pip-install-h22zu2rz
Processing /kaggle/input/nvidiaapex/repository/NVIDIA-apex-39e153a
  Created temporary directory: /tmp/pip-req-build-96342gtq
  Added file:///kaggle/input/nvidiaapex/repository/NVIDIA-apex-39e153a to build tracker '/tmp/pip-req-tracker-fm4u4rl0'
    Running setup.py (path:/tmp/pip-req-build-96342gtq/setup.py) egg_info for package from file:///kaggle/input/nvidiaapex/repository/NVIDIA-apex-39e153a
    Running command python setup.py egg_info
    torch.__version__  =  1.0.1.post2
    running egg_info
    creating p

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import datetime
import pkg_resources
import seaborn as sns
import time
import scipy.stats as stats
import gc
import re
import operator 
import sys
from sklearn import metrics
from sklearn import model_selection
import torch
import torch.nn as nn

import torch.utils.data

from torch.utils import data

import torch.nn.functional as F
from nltk.stem import PorterStemmer
from sklearn.metrics import roc_auc_score
%load_ext autoreload
%autoreload 2
%matplotlib inline
from tqdm import tqdm, tqdm_notebook
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
from apex import amp
import shutil

## Set device
device=torch.device('cuda')

## others
from IPython.core.debugger import set_trace
from joblib import Parallel, delayed

In [4]:
! pip install ../input/gptpytorchoriginal/gpt2-pytorch/pytorch-pretrained-BERT-master

Processing /kaggle/input/gptpytorchoriginal/gpt2-pytorch/pytorch-pretrained-BERT-master
  Stored in directory: /tmp/.cache/pip/wheels/06/a8/0f/ffe000aebc5034e7a616a8c0292ff34e45ea66acfc168a0760
Successfully built pytorch-pretrained-bert


In [5]:
GPT_MODEL_PATH = "../input/gpt2-models"

In [6]:
from pytorch_pretrained_bert import GPT2Tokenizer, GPT2ClassificationHeadModel
tokenizer = GPT2Tokenizer.from_pretrained(GPT_MODEL_PATH)
model = GPT2ClassificationHeadModel.from_pretrained(GPT_MODEL_PATH, clf_dropout=0.4, n_class=7)

/opt/conda/lib/python3.6/site-packages/pytorch_pretrained_bert/tokenization_gpt2.py:146: ResourceWarning: unclosed file <_io.TextIOWrapper name='../input/gpt2-models/vocab.json' mode='r' encoding='UTF-8'>
  self.encoder = json.load(open(vocab_file))
/opt/conda/lib/python3.6/site-packages/pytorch_pretrained_bert/tokenization_gpt2.py:151: ResourceWarning: unclosed file <_io.TextIOWrapper name='../input/gpt2-models/merges.txt' mode='r' encoding='utf-8'>
  bpe_data = open(merges_file, encoding='utf-8').read().split('\n')[1:-1]


In [7]:
os.listdir('../input')

['gptpytorchoriginal',
 'nvidiaapex',
 'gpt2oneepoch',
 'jigsaw-unintended-bias-in-toxicity-classification',
 'bert-pretrained-models',
 'gpt2-models',
 'gptbase']

In [8]:
MAX_SEQUENCE_LENGTH = 220
SEED = 7879
EPOCHS = 1
Data_dir="../input/jigsaw-unintended-bias-in-toxicity-classification"
Input_dir = "../input"
WORK_DIR = "../working/"
num_to_load= 1600000                         #Train size to match time limit
valid_size=  100000                          #Validation Size

#num_to_load= 200000                        #Train size to match time limit
#valid_size= 10000 

TOXICITY_COLUMN = 'target'

In [9]:
## Batch sampler 
class LenMatchBatchSampler(data.BatchSampler):
    def __iter__(self):

        buckets = [[]] * 100
        yielded = 0

        for idx in self.sampler:
            count_zeros = torch.sum(self.sampler.data_source[idx][0] == 0)
            count_zeros = int(count_zeros / 64) 
            if len(buckets[count_zeros]) == 0:  buckets[count_zeros] = []

            buckets[count_zeros].append(idx)

            if len(buckets[count_zeros]) == self.batch_size:
                batch = list(buckets[count_zeros])
                yield batch
                yielded += 1
                buckets[count_zeros] = []

        batch = []
        leftover = [idx for bucket in buckets for idx in bucket]

        for idx in leftover:
            batch.append(idx)
            if len(batch) == self.batch_size:
                yielded += 1
                yield batch
                batch = []

        if len(batch) > 0 and not self.drop_last:
            yielded += 1
            yield batch

        assert len(self) == yielded, "produced an inccorect number of batches. expected %i, but yielded %i" %(len(self), yielded)

def trim_tensors(tsrs):
    #set_trace()
    max_len = torch.max(torch.sum( (tsrs != 0  ), 1))
    if max_len > 2: 
        tsrs = tsrs[:, :max_len]
    return tsrs 

In [10]:
%%time
# tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)
train_df = pd.read_csv(os.path.join(Data_dir,"train.csv")).sample(num_to_load+valid_size,random_state=SEED)
print('loaded %d records' % len(train_df))

loaded 1700000 records
CPU times: user 13 s, sys: 2.41 s, total: 15.4 s
Wall time: 15.5 s


In [11]:
# Converting the lines to BERT format
# Thanks to https://www.kaggle.com/httpwwwfszyc/bert-in-keras-taming
def convert_lines(example, max_seq_length,tokenizer):
    #max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm_notebook(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(tokens_a)+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    print(longer)
    return np.array(all_tokens)

In [12]:
tokenizer.tokenize('Thisis \n cool!')

['This', 'is', 'Ġ', 'Ċ', 'Ġcool', '!']

In [13]:
sequences = convert_lines(train_df["comment_text"].fillna("DUMMY_VALUE"),MAX_SEQUENCE_LENGTH,tokenizer)
train_df=train_df.fillna(0)

In [14]:
# List all identities
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
y_columns = ['target']
train_df = train_df.drop(['comment_text'],axis=1)

In [15]:
gc.collect()

7

In [16]:
X = sequences[:num_to_load]                
X_val = sequences[num_to_load:]
y = train_df[y_columns].values[:num_to_load]
y_val = train_df[y_columns].values[num_to_load:]

test_df=train_df.tail(valid_size).copy()
train_df=train_df.head(num_to_load)

y_aux_train = train_df[['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']]
y_aux_test = test_df[['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']]

## Loss

In [17]:
# Overall
weights = np.ones((len(X),)) 

# Different Subgroups
weights += (train_df[identity_columns].values>=0.5).sum(axis=1).astype(bool).astype(np.int)

# Additional weights for a few subgroups
weights += (train_df['homosexual_gay_or_lesbian'].values>=0.5).astype(bool).astype(np.int)*0.5
weights += (train_df['black'].values>=0.5).astype(bool).astype(np.int)
weights += (train_df['white'].values>=0.5).astype(bool).astype(np.int)
weights += (train_df['muslim'].values>=0.5).astype(bool).astype(np.int)*0.25

## Identity with negatives
weights += ((train_df['target'].values<0.5).astype(bool).astype(np.int) \
* (train_df[identity_columns].values>=0.5).sum(axis=1).astype(bool).astype(np.int))*3.0

## Identity with positive
weights += ((train_df['target'].values>=0.5).astype(bool).astype(np.int) \
* (train_df['christian'].values>=0.5).astype(bool).astype(np.int))*3.0

## Identity with positive
weights += ((train_df['target'].values>=0.5).astype(bool).astype(np.int) \
* (train_df['jewish'].values>=0.5).astype(bool).astype(np.int))*1.25

## Non-identity with toxic
weights += ((train_df['target'].values>=0.5).astype(bool).astype(np.int) \
* (train_df[identity_columns].values<0.5).sum(axis=1).astype(bool).astype(np.int))*0.5

loss_weight = 1.0 / weights.mean()



def custom_loss(data, targets):
    bce_loss_1 = nn.BCEWithLogitsLoss(weight=targets[:,1:2])(data[:,:1],targets[:,:1])
    bce_loss_2 = nn.BCEWithLogitsLoss()(data[:,1:],targets[:,2:])
    return (bce_loss_1 * loss_weight) + bce_loss_2

## Data Loader

In [18]:
y_train = np.vstack([(train_df['target'].values>=0.5).astype(np.int),weights]).T
y_train_torch = torch.tensor(np.hstack([y_train, y_aux_train]), dtype=torch.float32)
train_dataset = torch.utils.data.TensorDataset(torch.tensor(X,dtype=torch.long), y_train_torch)

In [19]:
output_model_file = "gpt_pytorch.bin"
from pytorch_pretrained_bert import OpenAIAdam

# Load model
#model.load_state_dict(torch.load("../input/gpt2oneepoch/gpt_pytorch.bin"))

lr=2e-5
batch_size = 16
accumulation_steps=8
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

#n_output = y_train_torch.size(-1) - 1 #-1 not to include the weight

#model = BertForSequenceClassification.from_pretrained("../working",cache_dir=None,num_labels= n_output)

model.zero_grad()
model = model.to(device)

#param_optimizer = list(model.named_parameters())
#no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
#optimizer_grouped_parameters = [
#    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
#    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
#    ]

train = train_dataset

num_train_optimization_steps = int(EPOCHS*len(train)/batch_size/accumulation_steps)

optimizer = OpenAIAdam(model.parameters(),
                     lr=lr,
                     warmup=0.1,
                     t_total=num_train_optimization_steps)

model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)
model=model.train()

tq = tqdm_notebook(range(EPOCHS))
for epoch in tq:
    
    ran_sampler = torch.utils.data.RandomSampler(train_dataset)
    len_sampler = LenMatchBatchSampler(ran_sampler, batch_size = batch_size, drop_last = False)
    train_loader = torch.utils.data.DataLoader(train, batch_sampler = len_sampler) 

    #train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    
    avg_loss = 0.
    avg_accuracy = 0.
    lossf=None
    tk0 = tqdm_notebook(enumerate(train_loader),total=len(train_loader),leave=False)
    optimizer.zero_grad()   # Bug fix - thanks to @chinhuic
    for i,(x_batch, y_batch) in tk0:
#        optimizer.zero_grad()
        x_batch = trim_tensors(x_batch)
        y_pred = model(x_batch.to(device))
        loss =  custom_loss(y_pred,y_batch.to(device))
        
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        if (i+1) % accumulation_steps == 0:             # Wait for several backward steps
            optimizer.step()                            # Now we can do an optimizer step
            optimizer.zero_grad()
        if lossf:
            lossf = 0.98*lossf+0.02*loss.item()
        else:
            lossf = loss.item()
        tk0.set_postfix(loss = lossf)
        avg_loss += loss.item() / len(train_loader)
        avg_accuracy += torch.mean(((torch.sigmoid(y_pred[:,0])>0.5) == (y_batch[:,0]>0.5).to(device)).to(torch.float) ).item()/len(train_loader)
    tq.set_postfix(avg_loss=avg_loss,avg_accuracy=avg_accuracy)


torch.save(model.state_dict(), output_model_file)


In [20]:
# Run validation
# The following 2 lines are not needed but show how to download the model for prediction
#model = BertForSequenceClassification(bert_config,num_labels=n_output)
#model.load_state_dict(torch.load(output_model_file ))
#model.to(device)
for param in model.parameters():
    param.requires_grad=False
model.eval()
valid_preds = np.zeros((len(X_val)))
valid = torch.utils.data.TensorDataset(torch.tensor(X_val,dtype=torch.long))
valid_loader = torch.utils.data.DataLoader(valid, batch_size=32, shuffle=False)

tk0 = tqdm_notebook(valid_loader)
for i,(x_batch,)  in enumerate(tk0):
    pred = model(x_batch.to(device))
    valid_preds[i*32:(i+1)*32]=pred[:,0].detach().cpu().squeeze().numpy()




GPT2ClassificationHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (h): ModuleList(
      (0): Block(
        (ln_1): FusedLayerNorm(torch.Size([768]), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
        )
        (ln_2): FusedLayerNorm(torch.Size([768]), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
        )
      )
      (1): Block(
        (ln_1): FusedLayerNorm(torch.Size([768]), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
        )
        (ln_2): FusedLayerNorm(torch.Size([768]), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
        )
      )
      (2): Block(
        (ln_1): FusedLayerNorm(torch.Size([768]), eps=1e-05, elementwise_affine=T

In [21]:
# From baseline kernel

def calculate_overall_auc(df, model_name):
    true_labels = df[TOXICITY_COLUMN]>0.5
    predicted_labels = df[model_name]
    return metrics.roc_auc_score(true_labels, predicted_labels)

def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC], POWER),
        power_mean(bias_df[BPSN_AUC], POWER),
        power_mean(bias_df[BNSP_AUC], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)



SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive

def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan

def compute_subgroup_auc(df, subgroup, label, model_name):
    subgroup_examples = df[df[subgroup]>0.5]
    return compute_auc((subgroup_examples[label]>0.5), subgroup_examples[model_name])

def compute_bpsn_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df[(df[subgroup]>0.5) & (df[label]<=0.5)]
    non_subgroup_positive_examples = df[(df[subgroup]<=0.5) & (df[label]>0.5)]
    examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
    return compute_auc(examples[label]>0.5, examples[model_name])

def compute_bnsp_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df[(df[subgroup]>0.5) & (df[label]>0.5)]
    non_subgroup_negative_examples = df[(df[subgroup]<=0.5) & (df[label]<=0.5)]
    examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    return compute_auc(examples[label]>0.5, examples[model_name])

def compute_bias_metrics_for_model(dataset,
                                   subgroups,
                                   model,
                                   label_col,
                                   include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model."""
    records = []
    for subgroup in subgroups:
        record = {
            'subgroup': subgroup,
            'subgroup_size': len(dataset[dataset[subgroup]>0.5])
        }
        record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
        record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
        record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)


In [22]:

MODEL_NAME = 'model1'
test_df[MODEL_NAME]=torch.sigmoid(torch.tensor(valid_preds)).numpy()
TOXICITY_COLUMN = 'target'
bias_metrics_df = compute_bias_metrics_for_model(test_df, identity_columns, MODEL_NAME, 'target')
bias_metrics_df
get_final_metric(bias_metrics_df, calculate_overall_auc(test_df, MODEL_NAME))

,bnsp_auc,bpsn_auc,subgroup,subgroup_auc,subgroup_size
6,0.966778,0.911349,black,0.868885,771
2,0.971088,0.902181,homosexual_gay_or_lesbian,0.876601,595
7,0.967922,0.917773,white,0.882935,1360
5,0.961525,0.932858,muslim,0.890096,1132
4,0.959288,0.944718,jewish,0.909256,415
1,0.958181,0.965111,female,0.936812,2823
0,0.963263,0.958120,male,0.937248,2202
8,0.977514,0.947935,psychiatric_or_mental_illness,0.953811,226
3,0.975256,0.967436,christian,0.969938,1992


0.947128258747605